# Neuromast segmentation pipeline

**Caleb Reagor, Rockefeller University**

Notebook summary:
* Segment images from stacks using Cellpose
* Link masks using a particle tracking algorithm
* Optionally segment early frames manually
* Compute & save cells' normalized intensities

In [ ]:
# script dependencies
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib

# custom class for image stacks
# * additional dependencies:
#   * cellpose
#   * trackpy
#   * tqdm
#   * pims
#   * skimage
#   * cv2
#   * h5py
#   * PIL

from idataset import idataset
s = '/Volumes/LabDrive/'

In [ ]:
%matplotlib inline
matplotlib.rcParams['figure.dpi']= 1000
from IPython.display import Markdown

## Load all stacks from an imaging experiment

In [ ]:
mov = idataset(date = '122618', exper_id = 's7', 
               frames_directory = s + 'Raw/', 
               times_directory = s + 'Timepoints/',
               masks_directory = s + 'Masks/',
               results_directory = s + 'Results/')

In [ ]:
mov.show_frames_at_depth(19)

In [ ]:
mov.crop_z(zmin = 15, zmax = 23)

## Segment images using Cellpose

In [ ]:
# optionally, specify cell diameter in pixels (default 90)
# optionally, load previous segmentation results

mov.segment_frames(load_prev = True)

## Link masks using Trackpy

In [ ]:
# specify parameters for linking masks (first in z, then in time):

# * searchRange_t[_z]: the maximum distance features can move
#   between frames, in pixels (optional in z)
#
# * threshold_t[_z]: minimum number of points to survive
#   (optional in z)
#
# * memory_t[_z]: the maximum number of frames during which a
#   feature can vanish, then reappear nearby, and be considered
#   the same particle (optional)


mov.link_masks(searchRange_t = 50, threshold_t = 10, memory_t = 15)

In [ ]:
mov.show_segmentation_at_depth(4)

## Optionally, segment some frames manually

In [ ]:
%matplotlib notebook
mov.draw_masks_manually(t = , z = )

## Apply masks to frames to compute cells' per pixel intensities

In [ ]:
# optionally, stitch together specified cells' tracks & average the intensities

mov.cell_per_pixel_intensities(combine=[7,8])

In [ ]:
%matplotlib inline
matplotlib.rcParams['figure.dpi']= 1000

## Plot intensities, and optionally normalize to mature cells

In [ ]:
mov.plot_intensities([0,1,2,3,4,5,6], normalize_by=None)

In [ ]:
mov.plot_intensities([0], normalize_by=[1,2,3,4,5,6])

In [ ]:
mov.save_intensities_to_csv(cell = 0, normalize_by=[1,2,3,4,5,6])

In [ ]:
# to-do: 
# * manually validate fluorescence results:
#   * save_[load_]manual_masks as 4d np arrays
#   * option to save manual results separately
#   * manually segment at least one stack
#
# * determine when cells start to fluoresce:
#   * select region using modified manual selection code
#   * display region on scrollable frame to check accuracy
#   * select both region of interest and control region
#   * compare selections to determine when cell is fluorescent

In [ ]:
# import pims
# z = 4

# frames_list, size, border = [], 200, 20
# for i in range(mov.n_frames):

#     frame = mov.frames[i][z]
#     frame = np.clip(frame, mov.one[z], mov.ninety_nine[z])
#     frame -= mov.one[z]
#     intensity = mov.ninety_nine[z] - mov.one[z]

#     # generate the timestamp as a square array (size x size)
#     timestamp = mov.text_stamp("{:03d}m".format(mov.timepoints[i]), size, intensity)

#     # select the indices where text exists -> frame corner
#     idxs = np.stack(list(timestamp.nonzero()), axis=0).T; idxs_frame = idxs.copy()
#     idxs_frame[:,0] += frame.shape[0] - timestamp.shape[0] - border
#     idxs_frame[:,1] += frame.shape[1] - timestamp.shape[1] - border
#     frame[idxs_frame[:,0], idxs_frame[:,1]] = timestamp[idxs[:,0], idxs[:,1]]

#     frames_list.append(frame)
# pims.Frame(frames_list)